In [56]:
import sys
import re
from enum import Enum
from dataclasses import dataclass
from typing import List
import heapq

In [57]:
class CellType(Enum):
    EMPTY = 0
    BLOCK = 1
    SOURCE = 2
    TARGET = 3
    VIA = 4

@dataclass
class Cell:
    kind: CellType
    cost: int = 0

@dataclass
class Coord:
    row: int
    col: int
    layer: int

In [60]:
class MazeRouter:
    NUM_LAYERS = 2

    def __init__(self, in_path: str):
        self.in_path = in_path
        self.min_cost = 0

        self.cols = 0
        self.rows = 0
        self.Direction = 0
        self.via = 0
        self.via_count = 0

        self.grid: List[List[List[Cell]]] = []
        self.nets: List[List[Coord]] = []
        self.same_layer_nets: List[List[Coord]] = []
        
        self.load_config()

    def load_config(self):
        with open(self.in_path, 'r') as f:
            header = f.readline().strip()
            r, c, b, v = map(int, header.split(','))
            self.rows, self.cols = r, c
            self.Direction, self.via = b, v

            self.grid = [
                [
                    [Cell(CellType.EMPTY) for j in range(self.cols)]
                    for i in range(self.rows)
                ]
                for layer_idx in range(self.NUM_LAYERS)
            ]

            for line in f:
                line = line.strip()
                if line.startswith("OBS"):
                    self.add_obstacle(line)
                elif line.startswith("net"):
                    self.add_net(line)

    def add_obstacle(self, text: str):
        m = re.match(r"OBS\s*\((\d+),\s*(\d+),\s*(\d+)\)", text)
        if not m:
            print("Incorrect Format.", file=sys.stderr)
            return
        layer, row, col = map(int, m.groups())
        if 0 <= layer < self.NUM_LAYERS and 0 <= row < self.rows and 0 <= col < self.cols:
            self.grid[layer][row][col].kind = CellType.BLOCK
        else:
            print("Obstacle out of Range", file=sys.stderr)

    def add_net(self, text: str):
        path = []
        for m in re.finditer(r"\((\d+),\s*(\d+),\s*(\d+)\)", text):
            L, r, c = map(int, m.groups())
            path.append(Coord(r,c,L))
        if len(path) >= 2:
            self.nets.append(path)

    def categorize_nets(self):
        self.same_layer_nets = [
            net for net in self.nets
            if len({p.layer for p in net}) == 1
        ]

    def set_SourceAndTarget_flags(self, net: List[Coord]):
        for i,p in enumerate(net):
            self.grid[p.layer][p.row][p.col].kind = (
                CellType.SOURCE if i==0 else CellType.TARGET
            )
            

    def back_propagate(self, src: Coord, tgt: Coord) -> List[Coord]:
        L = src.layer
        path = [tgt]
        cur = tgt
        max_iters = self.rows * self.cols * 2
        iters = 0

        while (cur.row, cur.col) != (src.row, src.col) and iters < max_iters:
            iters += 1
            best = None
            best_cost = float('inf')

            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = cur.row+dr, cur.col+dc
                if 0 <= nr < self.rows and 0 <= nc < self.cols:
                    cell = self.grid[L][nr][nc]
                    if cell.cost >= 0 and cell.cost < best_cost:
                        best_cost = cell.cost
                        best = Coord(nr, nc, L)

            if not best:
                break

            cur = best
            path.insert(0, cur)

        if (cur.row, cur.col) != (src.row, src.col):
            print("aborting back-propagation", file=sys.stderr)
        return path

    def _lee_wave(self, src: Coord, tgt: Coord, L: int) -> bool:
        INF = float('inf')
        for row_idx in range(self.rows):
            for col_idx in range(self.cols):
                cell = self.grid[L][row_idx][col_idx]
                cell.cost = -1 if cell.kind == CellType.BLOCK else INF

        self.grid[L][src.row][src.col].cost = 0
        pq = [(0, src.row, src.col)]

        while pq:
            acc, row_idx, col_idx = heapq.heappop(pq)
            if acc > self.grid[L][row_idx][col_idx].cost:
                continue
                
            if (row_idx, col_idx) == (tgt.row, tgt.col):
                min_cost = self.grid[L][row_idx][col_idx].cost
                self.min_cost = min_cost
                return True

            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = row_idx + dr, col_idx + dc
                if not (0 <= nr < self.rows and 0 <= nc < self.cols):
                    continue

                neigh = self.grid[L][nr][nc]
                if neigh.kind == CellType.BLOCK:
                    continue

                step = 1
                if not ((dr == 0 and L == 0) or (dc == 0 and L == 1)):
                    step = self.Direction

                new_cost = acc + step
                if new_cost < neigh.cost:
                    neigh.cost = new_cost
                    heapq.heappush(pq, (new_cost, nr, nc))

        return False
    
    def route_same_layer_multi_targets(self):
        self.categorize_nets()
        results = []

        for net_idx, net in enumerate(self.same_layer_nets):
            layer = net[0].layer

            connected: List[Coord] = [net[0]]
            targets: List[Coord] = net[1:].copy()
            segments: List[ (Coord, Coord, List[Coord], float) ] = []

            while targets:
                best_seg = None 
                
                for start in connected:
                    for tgt in targets:
                        if not self._lee_wave(start, tgt, layer):
                            continue
                        path = self.back_propagate(start, tgt)
                        cost = self.min_cost
                        if len(path) > 1 and (best_seg is None or cost < best_seg[0]):
                            best_seg = (cost, start, tgt, path)

                if best_seg is None:
                    break

                cost, start, chosen_tgt, path = best_seg
                segments.append((start, chosen_tgt, path, cost))

                for c in path:
                    if all(not (c.row==cc.row and c.col==cc.col) for cc in connected):
                        self.grid[layer][c.row][c.col].kind = CellType.BLOCK
                        connected.append(c)

                targets.remove(chosen_tgt)

            total_cost = sum(seg_cost for _,_,_,seg_cost in segments)
            results.append((net_idx, segments, total_cost))

        return results


In [64]:
def main():
    router = MazeRouter('input5.txt')
    print(f"Grid: {router.rows}×{router.cols}, DirPenalty={router.Direction}, viaCost={router.via}\n")

    print("Obstacles:")
    for L in range(MazeRouter.NUM_LAYERS):
        for r in range(router.rows):
            for c in range(router.cols):
                if router.grid[L][r][c].kind == CellType.BLOCK:
                    print(f"  layer{L}, row{r}, col{c}")
    print()

    print("Loaded nets:")
    for i, net in enumerate(router.same_layer_nets if (router.categorize_nets() or True) else []):
        pts = [(p.layer,p.row,p.col) for p in net]
        print(f"  net{i}: {pts}")
    print()

    results = router.route_same_layer_multi_targets()
    print("=== MULTI-TARGET ROUTES ===")
    for net_idx, segments, total_cost in results:
        print(f"net{net_idx}: total_cost = {total_cost}")
        for start, tgt, path, cost in segments:
            coords = [(c.layer,c.row,c.col) for c in path]
            print(f"  from {(start.layer,start.row,start.col)} → {(tgt.layer,tgt.row,tgt.col)}: path={coords}")
        if not segments:
            print("  (no reachable targets)")
        print()

            
if __name__ == "__main__":
    main()

Grid: 10×10, DirPenalty=5, viaCost=20

Obstacles:
  layer0, row0, col6
  layer0, row1, col6
  layer0, row2, col3
  layer0, row2, col6
  layer0, row3, col6
  layer0, row4, col6

Loaded nets:
  net0: [(0, 0, 1), (0, 2, 4), (0, 0, 8), (0, 7, 6)]
  net1: [(0, 9, 1), (0, 8, 3)]

=== MULTI-TARGET ROUTES ===
net0: total_cost = 67
  from (0, 0, 1) → (0, 2, 4): path=[(0, 0, 1), (0, 0, 2), (0, 0, 3), (0, 0, 4), (0, 1, 4), (0, 2, 4)]
  from (0, 2, 4) → (0, 7, 6): path=[(0, 2, 4), (0, 2, 5), (0, 3, 5), (0, 4, 5), (0, 5, 5), (0, 5, 6), (0, 6, 6), (0, 7, 6)]
  from (0, 5, 6) → (0, 0, 8): path=[(0, 5, 6), (0, 5, 7), (0, 5, 8), (0, 4, 8), (0, 3, 8), (0, 2, 8), (0, 1, 8), (0, 0, 8)]

net1: total_cost = 7
  from (0, 9, 1) → (0, 8, 3): path=[(0, 9, 1), (0, 9, 2), (0, 9, 3), (0, 8, 3)]

